In [2]:
%reload_ext autoreload
%autoreload 2
from exploration import *
item_data, train_len, test_len, domain2items =  restore_globals()

In [3]:
df_items, df_ds, df_ds_test, df_ds_valid = restore_globals_dfs()

## Dataset stats
### len

In [2]:
valid_len = len([None for _ in ds(VALID_SPLIT_DATA)])
train_len, test_len, valid_len, train_len/test_len, valid_len/train_len

(413163, 177070, 82632, 2.3333314508386516, 0.19999854778864515)

### Stats

In [56]:
# solves a type problem
for df in [df_ds,df_ds_valid,df_ds_test]:
    df['is_zero_view'] = (df['view_len']==0)
    for c in df.columns:
        if 'item_bought' not in c: df[c]=df[c].astype({c: np.int32}, copy=False)
    

In [84]:
for i in range(3):
    df = [df_ds,df_ds_valid,df_ds_test][i]
    dfn = ['train','valid','test'][i]
    df=df[[c for c in df.columns if 'is' in c]]
    df=pd.concat([df.sum().rename('total'),df.mean().rename('ratio')],axis=1)
    print(dfn)
    display(df)

train


,total,ratio
items_view_distinct,3355205,8.120778
domain_view_distinct,1320729,3.196629
category_view_distinct,1358097,3.287073
is_viewed_item,121422,0.293884
is_viewed_domain,203857,0.493406
is_viewed_category,202257,0.489533
item_is_last_viewed,63367,0.153370
item_is_most_common,64795,0.156827
domain_is_last_viewed,141844,0.343312
domain_is_most_common,146381,0.354294


valid


,total,ratio
items_view_distinct,673247,8.147534
domain_view_distinct,265286,3.210451
category_view_distinct,272877,3.302316
is_viewed_item,24275,0.293772
is_viewed_domain,40800,0.493755
is_viewed_category,40498,0.490101
item_is_last_viewed,12687,0.153536
item_is_most_common,12992,0.157227
domain_is_last_viewed,28566,0.345701
domain_is_most_common,29380,0.355552


test


,total,ratio
items_view_distinct,1448594,8.180912
domain_view_distinct,569854,3.218241
category_view_distinct,585989,3.309364
is_zero_view,11370,0.064212


In [5]:
def get_description(i):
    df = [df_ds,df_ds_valid,df_ds_test][i]
    df=df[[c for c in df.columns if 'is_' not in c and 'item_bought' not in c]]
    dfn = ['train','valid','test'][i]
    ddf = df.describe(percentiles=[.25, .5, .75,.9, .95,.99])    
    ddf = ddf.transpose()
    ddf['dataset']=dfn
    ddf.index.name = 'metrics'
    ddf = ddf.reset_index().set_index(['metrics','dataset'])
    return ddf
descriptions_df = pd.concat([get_description(i) for i in range(3) ],axis=0).sort_index()
descriptions_df.to_csv('description.csv')
descriptions_df

count       mean        std  \
metrics                               dataset                                   
category_view_distinct                test     177070.0   3.309364   3.625859   
                                      train    413163.0   3.287073   3.610937   
                                      valid     82632.0   3.302316   3.662051   
domain_view_distinct                  test     177070.0   3.218241   3.462768   
                                      train    413163.0   3.196629   3.451391   
                                      valid     82632.0   3.210451   3.495877   
items_view_distinct                   test     177070.0   8.180912  10.735862   
                                      train    413163.0   8.120778  10.719081   
                                      valid     82632.0   8.147534  10.813028   
items_view_distinct_in_guessed_domain train    413163.0   2.078514   4.147911   
                                      valid     82632.0   2.080163   4.148764   
searches_len                          test     177070.0  14.847970  20.604518   
                                      train    413163.0  14.689478  20.457799   
                                      valid     82632.0  14.801530  20.605561   
view_len                              test     177070.0  14.449788  19.325787   
                                      train    413163.0  14.352730  19.293081   
                                      valid     82632.0  14.414234  19.411456   

                                               min  25%  50%   75%   90%  \
metrics                               dataset                              
category_view_distinct                test     0.0  1.0  2.0   4.0   7.0   
                                      train    0.0  1.0  2.0   4.0   7.0   
                                      valid    0.0  1.0  2.0   4.0   7.0   
domain_view_distinct                  test     0.0  1.0  2.0   4.0   7.0   
                                      train    0.0  1.0  2.0   4.0   7.0   
                                      valid    0.0  1.0  2.0   4.0   7.0   
items_view_distinct                   test     0.0  2.0  4.0  10.0  20.0   
                                      train    0.0  2.0  4.0  10.0  20.0   
                                      valid    0.0  2.0  4.0  10.0  20.0   
items_view_distinct_in_guessed_domain train    0.0  0.0  0.0   2.0   6.0   
                                      valid    0.0  0.0  0.0   2.0   6.0   
searches_len                          test     0.0  2.0  7.0  20.0  41.0   
                                      train    0.0  1.0  7.0  19.0  40.0   
                                      valid    0.0  2.0  7.0  20.0  41.0   
view_len                              test     0.0  2.0  7.0  19.0  37.0   
                                      train    0.0  2.0  7.0  18.0  37.0   
                                      valid    0.0  2.0  7.0  18.0  37.0   

                                                95%   99%    max  
metrics                               dataset                     
category_view_distinct                test     10.0  18.0   84.0  
                                      train    10.0  18.0   80.0  
                                      valid    10.0  18.0   80.0  
domain_view_distinct                  test     10.0  17.0   72.0  
                                      train    10.0  17.0   69.0  
                                      valid    10.0  17.0   69.0  
items_view_distinct                   test     29.0  52.0  157.0  
                                      train    29.0  52.0  229.0  
                                      valid    29.0  52.0  187.0  
items_view_distinct_in_guessed_domain train     9.0  20.0  146.0  
                                      valid     9.0  20.0  146.0  
searches_len                          test     57.0  96.0  302.0  
                                      train    57.0  95.0  284.0  
                                      valid    57.0  95.0  2

### Stats x cantidad de dominios distintos

In [ ]:
dom_stats_lu(1,1)

In [6]:
dom_stats_lu(1,2)

Caso 1 <= |doms_view| =< 2
Stats on whole train
P(2_dom) = 0.5124563428961451
P(dom_inview | 2_dom) = 0.48935898889140783
P(domain_is_last_viewed | 2_dom) = 0.44033382452958514
P(domain_is_most_common | 2_dom) = 0.4346425602660017
P(domain_is_last_viewed | dom_inview ^ 2_dom) = 0.8998175869357501
P(domain_is_most_common | dom_inview ^ 2_dom) = 0.8881875476542066
Stats on valid
P(2_dom) = 0.5111458030787104
P(dom_inview | 2_dom) = 0.48964178327059216
P(domain_is_last_viewed | 2_dom) = 0.4422189075928688
P(domain_is_most_common | 2_dom) = 0.4357553803537183
P(domain_is_last_viewed | dom_inview ^ 2_dom) = 0.90314781683671
P(domain_is_most_common | dom_inview ^ 2_dom) = 0.8899472946182486


In [7]:
dom_stats_lu(3,4)

Caso 3 <= |doms_view| =< 4
Stats on whole train
P(2_dom) = 0.20483925230478045
P(dom_inview | 2_dom) = 0.5521552131581435
P(domain_is_last_viewed | 2_dom) = 0.3428726722752623
P(domain_is_most_common | 2_dom) = 0.31925276491161736
P(domain_is_last_viewed | dom_inview ^ 2_dom) = 0.6209715386261502
P(domain_is_most_common | dom_inview ^ 2_dom) = 0.5781938797346459
Stats on valid
P(2_dom) = 0.20523525994772002
P(dom_inview | 2_dom) = 0.5510348487528746
P(domain_is_last_viewed | 2_dom) = 0.340468187982782
P(domain_is_most_common | 2_dom) = 0.3211863907069992
P(domain_is_last_viewed | dom_inview ^ 2_dom) = 0.6178705189941145
P(domain_is_most_common | dom_inview ^ 2_dom) = 0.5828785446762975


In [8]:
dom_stats_lu(3,5)

Caso 3 <= |doms_view| =< 5
Stats on whole train
P(2_dom) = 0.26182886657324106
P(dom_inview | 2_dom) = 0.5560187838562369
P(domain_is_last_viewed | 2_dom) = 0.3338479173214517
P(domain_is_most_common | 2_dom) = 0.31167150437242325
P(domain_is_last_viewed | dom_inview ^ 2_dom) = 0.6004256097358227
P(domain_is_most_common | dom_inview ^ 2_dom) = 0.5605413223827495
Stats on valid
P(2_dom) = 0.26267063607319197
P(dom_inview | 2_dom) = 0.5550794747753973
P(domain_is_last_viewed | 2_dom) = 0.33245795899562314
P(domain_is_most_common | 2_dom) = 0.314351531905091
P(domain_is_last_viewed | dom_inview ^ 2_dom) = 0.598937583001328
P(domain_is_most_common | dom_inview ^ 2_dom) = 0.5663180610889774


In [9]:
dom_stats_lu(5,17)

Caso 5 <= |doms_view| =< 17
Stats on whole train
P(2_dom) = 0.20935078891381853
P(dom_inview | 2_dom) = 0.5925013873473918
P(domain_is_last_viewed | 2_dom) = 0.2715385682574917
P(domain_is_most_common | 2_dom) = 0.2560812060673326
P(domain_is_last_viewed | dom_inview ^ 2_dom) = 0.45829186910964115
P(domain_is_most_common | dom_inview ^ 2_dom) = 0.4322035551913208
Stats on valid
P(2_dom) = 0.2093498886629877
P(dom_inview | 2_dom) = 0.5945430371697786
P(domain_is_last_viewed | 2_dom) = 0.2764321637088849
P(domain_is_most_common | 2_dom) = 0.2640036996358171
P(domain_is_last_viewed | dom_inview ^ 2_dom) = 0.464948954788527
P(domain_is_most_common | dom_inview ^ 2_dom) = 0.4440447253281478


In [10]:
dom_stats_lu(6,17)

Caso 6 <= |doms_view| =< 17
Stats on whole train
P(2_dom) = 0.1523611746453579
P(dom_inview | 2_dom) = 0.6009531374106434
P(domain_is_last_viewed | 2_dom) = 0.2603653693407466
P(domain_is_most_common | 2_dom) = 0.24548054011119935
P(domain_is_last_viewed | dom_inview ^ 2_dom) = 0.43325403119217554
P(domain_is_most_common | dom_inview ^ 2_dom) = 0.4084853291038858
Stats on valid
P(2_dom) = 0.15191451253751573
P(dom_inview | 2_dom) = 0.6039990440532144
P(domain_is_last_viewed | 2_dom) = 0.26607185533338645
P(domain_is_most_common | 2_dom) = 0.25420218274516054
P(domain_is_last_viewed | dom_inview ^ 2_dom) = 0.4405170139804801
P(domain_is_most_common | dom_inview ^ 2_dom) = 0.42086520706937486


In [ ]:
dom_stats_lu(6,10)

Caso 6 <= |doms_view| =< 10
Stats on whole train


## Cantidad de dominios

In [4]:
# en item_data
len({i['domain_id'] for i in item_data.values()})

7894

In [10]:
# en item_bought del train
len({item_data[x['item_bought']]['domain_id'] for x in ds()})

3214

In [11]:
# en views del train
len({item_data[i['event_info']]['domain_id'] for x in ds() for i in x['user_history'] if i['event_type']=='view'})

7690

In [12]:
# en views del test
len({item_data[i['event_info']]['domain_id'] for x in ds(TEST_DATA) for i in x['user_history'] if i['event_type']=='view'})

7056

In [16]:
# en views del valid
len({item_data[i['event_info']]['domain_id'] for x in ds(VALID_SPLIT_DATA) for i in x['user_history'] if i['event_type']=='view'})

6258

## Cantidad de dominios

In [2]:
# en item_data
len(item_data)

2102277

In [3]:
# en item_bought del train
len({x['item_bought'] for x in ds()})

64928

In [4]:
# en views del train
len({i['event_info'] for x in ds() for i in x['user_history'] if i['event_type']=='view'})

1601277

In [5]:
# en views del train
len({i['event_info'] for x in ds(TEST_DATA) for i in x['user_history'] if i['event_type']=='view'})

832079

In [ ]:
# en views del train
len({i['event_info'] for x in ds(VALID_SPLIT_DATA) for i in x['user_history'] if i['event_type']=='view'})

## Heuristicas

In [6]:
HEURISTICS = [ oraculo_repeat, 
               most_common_heuristic_repeat, 
               most_common_heuristic, 
               most_common_heuristic_better_cheater,
               last_item_heuristic_repeat ]
data = []
for fn in [TRAIN_DATA, VALID_SPLIT_DATA]:
    print(f'Using {fn}')
    for f_heuristic in HEURISTICS:
        hn = f_heuristic.__name__
        start = time.time()
        result = eval_predictor(f_heuristic, fn=fn)
        end = time.time()
        print(f'{hn}: {result} {end-start}')
        data.append((fn,hn,result))
df_heu = pd.DataFrame(data,columns=('dataset','heuristic','value'))
df_heu.set_index(['dataset','heuristic'])

Using train_dataset.jl
oraculo_repeat: 0.7720239450259194 35.74330234527588
most_common_heuristic_repeat: 0.1334881421319703 41.394267559051514
most_common_heuristic: 0.23604181274071392 258.54558634757996
most_common_heuristic_better_cheater: 0.3235696976541462 267.8741343021393
last_item_heuristic_repeat: 0.13063950732147528 36.9194118976593
Using valid_split.jl
oraculo_repeat: 0.772023945030787 7.15475058555603
most_common_heuristic_repeat: 0.13383028360199165 8.283761501312256
most_common_heuristic: 0.23697937133689528 51.9269642829895
most_common_heuristic_better_cheater: 0.3236328112846593 53.868932247161865
last_item_heuristic_repeat: 0.13091217908843322 7.363302707672119


value
dataset          heuristic                                     
train_dataset.jl oraculo_repeat                        0.772024
                 most_common_heuristic_repeat          0.133488
                 most_common_heuristic                 0.236042
                 most_common_heuristic_better_cheater  0.323570
                 last_item_heuristic_repeat            0.130640
valid_split.jl   oraculo_repeat                        0.772024
                 most_common_heuristic_repeat          0.133830
                 most_common_heuristic                 0.236979
                 most_common_heuristic_better_cheater  0.323633
                 last_item_heuristic_repeat            0.130912

In [2]:
HEURISTICS = [ oraculo_repeat, 
               most_common_heuristic_repeat, 
               most_common_heuristic, 
               most_common_heuristic_better_cheater,
               last_item_heuristic_repeat ]
data = []
for fn in [TRAIN_DATA, VALID_SPLIT_DATA]:
    print(f'Using {fn}')
    for f_heuristic in HEURISTICS:
        hn = f_heuristic.__name__
        start = time.time()
        result = eval_predictor(f_heuristic, fn=fn)
        end = time.time()
        print(f'{hn}: {result} {end-start}')
        data.append((fn,hn,result))
df_heu = pd.DataFrame(data,columns=('dataset','heuristic','value'))
df_heu.set_index(['dataset','heuristic'])

Using train_dataset.jl
oraculo_repeat: 0.7720239450259194 34.94313979148865
most_common_heuristic_repeat: 0.1334881421319703 40.901962995529175
most_common_heuristic: 0.23604585157550667 258.47818779945374
most_common_heuristic_better_cheater: 0.3235358657416154 267.09407210350037
last_item_heuristic_repeat: 0.13063950732147528 36.460758447647095
Using valid_split.jl
oraculo_repeat: 0.772023945030787 7.032893180847168
most_common_heuristic_repeat: 0.13383028360199165 8.200316905975342
most_common_heuristic: 0.236993912980349 51.69042706489563
most_common_heuristic_better_cheater: 0.32360728991626075 53.7637255191803
last_item_heuristic_repeat: 0.13091217908843322 7.348647594451904


value
dataset          heuristic                                     
train_dataset.jl oraculo_repeat                        0.772024
                 most_common_heuristic_repeat          0.133488
                 most_common_heuristic                 0.236046
                 most_common_heuristic_better_cheater  0.323536
                 last_item_heuristic_repeat            0.130640
valid_split.jl   oraculo_repeat                        0.772024
                 most_common_heuristic_repeat          0.133830
                 most_common_heuristic                 0.236994
                 most_common_heuristic_better_cheater  0.323607
                 last_item_heuristic_repeat            0.130912

In [45]:
df_heu.set_index(['heuristic','dataset']).sort_index()

value
heuristic                            dataset                   
last_item_heuristic_repeat           train_dataset.jl  0.130640
                                     valid_split.jl    0.130912
most_common_heuristic                train_dataset.jl  0.236022
                                     valid_split.jl    0.236989
most_common_heuristic_better_cheater train_dataset.jl  0.323553
                                     valid_split.jl    0.323642
most_common_heuristic_repeat         train_dataset.jl  0.133488
                                     valid_split.jl    0.133830
oraculo_repeat                       train_dataset.jl  0.772024
                                     valid_split.jl    0.772024

In [51]:
# pegarle al item en el puesto 1 + todos 0's
withcoefs(np.array([12]+[0]*9))/idcg(1)

0.7720239450300733

In [52]:
# pegarle al dominio pero nunca al item
withcoefs(np.array([1]*10))/idcg(1)

0.29231138372243276

In [55]:
# pegarle al item en el puesto random + todos 0's
avg = 0
for i in range(10):
    score = np.zeros(10)
    score[i]=12
    avg += withcoefs(score/idcg(1))
avg /= 10
avg

0.3507736604669193

### Combinador

In [ ]:
%%time
# 51s
solver = CombinerHeuristic(cb_fit_casitos, cb_proc_casitos)
solver.fit(lambda : ds(TRAIN_SPLIT_DATA))


In [ ]:
%%time
# 2min
eval_predictor(solver, fn=VALID_SPLIT_DATA)

In [ ]:
# Lo de arriba + cheteado para adivinar el dominio en el caso de que esté en views se logra 0.34
def cb_proc_casitos_cheater(self, item):
    self.iviews = views(item)
    iids = [v['event_info'] for v in self.iviews]
    icount = Counter(iids)
    dids = [dom(iid) for iid in iids]
    dcount = Counter(dids)
    views_len = len(self.iviews)
    if views_len>0:
        proba_domain_in_view = 0.5275938818
        proba_item_in_view = 0.5956234027
        dbought=dom(item['item_bought'])
        if dbought in set(dids):
            p_items_inview = { k: proba_item_in_view*v/dcount[dom(k)] for k,v in icount.items() if dom(k)==dbought}
            self.p_dom = { dbought: proba_item_in_view }
            sdoms = {dbought}
        else:
            p_items_inview = { k: proba_item_in_view*v/dcount[dom(k)] for k,v in icount.items() }
            self.p_dom = { k: (1-proba_domain_in_view)*v/views_len for k,v in dcount.items() }
            sdoms = set(dids)
        self.p_dom[''] = (1-proba_domain_in_view)
        self.p_items = {}
        for d in sdoms:
            td = self.boughts_totals.get(d,0)
            if td>0:
                for i, cant in self.boughts[d].items():
                    self.p_items[i] = (1-proba_item_in_view)*cant/td + p_items_inview.get(i,0)
            if td<10:
                for i in domain2items[d][:10-td]:
                    self.p_items[i] = 0
        for i,pi in p_items_inview.items():
            if i not in self.p_items:
                self.p_items[i]=pi
        self.p_items=list(self.p_items.items())

## Frecuencia de dominios en items comprados

In [11]:
for fn in [TRAIN_DATA, TRAIN_SPLIT_DATA, VALID_SPLIT_DATA]:
    print(fn)
    l = [ dom(x['item_bought']) for x in ds(fn) ]
    cs=Counter(l)
    for k,v in cs.most_common(20):
        print(f'{k}: {v/len(l)}')

train_dataset.jl
MLB-CELLPHONES: 0.06067823110975571
MLB-SNEAKERS: 0.035356505785852074
MLB-SUPPLEMENTS: 0.023143408291642766
MLB-HEADPHONES: 0.021911448992286336
MLB-SMARTWATCHES: 0.01927326503099261
MLB-DOLLS: 0.011896031348402447
MLB-VIDEO_GAMES: 0.011223173420659642
MLM-HEADPHONES: 0.011140881443885343
MLB-BOOTS_AND_BOOTIES: 0.01046802351614254
MLB-STREAMING_MEDIA_DEVICES: 0.00993312566710959
MLB-T_SHIRTS: 0.009066639558721376
MLB-SURVEILLANCE_CAMERAS: 0.008333272824526882
MLB-PANTS: 0.008333272824526882
MLB-HAIR_TREATMENTS: 0.00824129943872031
MLB-CELLPHONE_SCREENS: 0.007374813330332097
MLB-WALL_AND_CEILING_LIGHTS: 0.0071884462064608886
MLB-SHORTS: 0.006912526049041178
MLB-HARD_DRIVES_AND_SSDS: 0.0068810614696862985
MLB-SPEAKERS: 0.006714057163879631
MLM-CELLPHONE_COVERS: 0.0065301103922664906
train_split.jl
MLB-CELLPHONES: 0.06081124009548272
MLB-SNEAKERS: 0.0351434509924939
MLB-SUPPLEMENTS: 0.023138525584589644
MLB-HEADPHONES: 0.02201306382759862
MLB-SMARTWATCHES: 0.019205460304

In [12]:
for fn in [TRAIN_DATA, TRAIN_SPLIT_DATA, VALID_SPLIT_DATA]:
    print(fn)
    l = [ dom(x['item_bought']) for x in ds(fn) if view_len(x)==0 ]
    cs=Counter(l)
    for k,v in cs.most_common(20):
        print(f'{k}: {v/len(l)}')


train_dataset.jl
MLB-CELLPHONES: 0.04004033914764875
MLB-SUPPLEMENTS: 0.028312105479400888
MLB-SNEAKERS: 0.02745303103873305
MLB-HEADPHONES: 0.019982818511186645
MLB-SMARTWATCHES: 0.015612744182571994
MLB-VIDEO_GAMES: 0.014716318679266425
MLB-BOOTS_AND_BOOTIES: 0.010196840100100848
MLB-CELLPHONE_SCREENS: 0.00986068053636126
MLB-DOLLS: 0.009375116722070743
MLB-STREAMING_MEDIA_DEVICES: 0.009300414596795278
MLB-HAIR_TREATMENTS: 0.00915101034624435
MLB-T_SHIRTS: 0.009001606095693422
MLM-HEADPHONES: 0.008105180592387854
MLB-SURVEILLANCE_CAMERAS: 0.007731669966010533
MLB-HARD_DRIVES_AND_SSDS: 0.007171404026444552
MLB-WALL_AND_CEILING_LIGHTS: 0.006872595525342696
MLB-FLEA_AND_TICK_TREATMENTS: 0.006499084898965376
MLB-PANTS: 0.006088223209950323
MLB-HAIR_CLIPPERS: 0.006088223209950323
MLM-CELLPHONE_COVERS: 0.005901467896761663
train_split.jl
MLB-CELLPHONES: 0.03961876284806578
MLB-SUPPLEMENTS: 0.02868622687348159
MLB-SNEAKERS: 0.027144458979629974
MLB-HEADPHONES: 0.020183143337693887
MLB-SMART

## Dominio vs Categoria

In [5]:
df_items[df_items['domain_id']=='MLB-GARDENING'].sort_values('category_id')

,Unnamed: 0,item_id,title,domain_id,product_id,price,category_id,condition
15080,15080,527365,Kit 2 Pçs * Irrigador Por Gotejamento Para Va...,MLB-GARDENING,NaN,26.99,MLB11032,new
1579089,1579089,891169,Forração Jardim Pedrisco ( Cascalho) Branco Sa...,MLB-GARDENING,NaN,25.00,MLB11032,new
1578926,1578926,447640,Vaso Plantas Cuia Coco Com Suporte E Corrente ...,MLB-GARDENING,NaN,137.90,MLB11032,new
1578587,1578587,1276751,Kit 10 Vaso De Feltro 1 Litro Grow Cultivo Ind...,MLB-GARDENING,NaN,89.90,MLB11032,new
1577667,1577667,1042607,Pedra Preto Fosco Seixo Rolado Dolomita Jardim...,MLB-GARDENING,NaN,35.00,MLB11032,new
1571068,1571068,1145441,3 Sacas Substrato Carolina Soil Padrão Ce 07 R...,MLB-GARDENING,NaN,229.00,MLB11032,new
1566826,1566826,1490107,345 Células Espuma Fenólica P/ Germinação De S...,MLB-GARDENING,NaN,27.99,MLB11032,new
1565964,1565964,1848724,3 Mudas De Costela De Adão (frete Grátis),MLB-GARDENING,NaN,35.00,MLB11032,new
1560656,1560656,502435,Kokedama Asplenium Nidus - Planta Especial Par...,MLB-GARDENING,NaN,119.00,MLB11032,new
1548854,1548854,725906,Muda Orquídea Catasetum Hookeri,MLB-GARDENING,NaN,35.00,MLB11032,new


In [6]:
s=df_items.groupby('domain_id')['category_id'].nunique()
s

domain_id
MLB-3D_GLASSES              1
MLB-3D_PENS                 1
MLB-3D_PRINTERS             1
MLB-3D_PRINTER_FILAMENTS    1
MLB-3D_PRINTER_HEATBEDS     1
                           ..
MLM-YARNS                   1
MLM-YERBA_MATE              1
MLM-YOGA_MATS               1
MLM-ZAFUS                   1
MLM-ZIPPERS                 1
Name: category_id, Length: 7893, dtype: int64

In [7]:
df_items[df_items['category_id']=='MLB443874']

,Unnamed: 0,item_id,title,domain_id,product_id,price,category_id,condition
46778,46778,2070887,Protetor Motor Slider Titan 160 2016 Ex 2019 /...,MLB-MOTORCYCLE_CRASH_BARS,NaN,139.90,MLB443874,new
48955,48955,76813,Slider Dianteiro Nylon Gp Racing Fazer 250 Fz2...,MLB-VEHICLE_ACCESSORIES,NaN,379.90,MLB443874,new
92660,92660,622541,Protetor Motor Slider Fazer 150 E Slider Balança,MLB-MOTORCYCLE_CRASH_BARS,NaN,120.00,MLB443874,new
96738,96738,994064,Slider P/ Moto Hornet Dianteiro Hornet Kit 2 M...,MLB-MOTORCYCLE_CRASH_BARS,NaN,285.00,MLB443874,new
129173,129173,1050521,Slider Er6n Er6 N Ern6 2013 ...,MLB-MOTORCYCLE_CRASH_BARS,NaN,249.00,MLB443874,new
...,...,...,...,...,...,...,...,...
2085723,2085723,1137023,Slider Fazer 250 2018 2019 Slider Retrovisor M...,MLB-MOTORCYCLE_CRASH_BARS,NaN,459.99,MLB443874,new
2085755,2085755,1377335,Meio Slider Reposição Procton Racing Honda Cb ...,MLB-MOTORCYCLE_CRASH_BARS,NaN,270.00,MLB443874,new
2086920,2086920,1417697,Slider Esportivo Protetor Super Racing Factor ...,MLB-MOTORCYCLE_CRASH_BARS,NaN,124.90,MLB443874,new
2087580,2087580,1195040,Protetor Motor Mata Cachorro Slider Fazer 250...,MLB-MOTORCYCLE_CRASH_BARS,NaN,130.00,MLB443874,new


In [8]:
s=df_items.groupby('category_id')['domain_id'].nunique()
s

category_id
MLB100074    1
MLB100075    1
MLB10015     1
MLB100177    1
MLB10018     1
            ..
MLM9883      1
MLM9893      1
MLM9905      1
MLM9933      1
MLM9944      1
Name: domain_id, Length: 11493, dtype: int64

In [9]:
s[s==8]

category_id
MLB443874    8
Name: domain_id, dtype: int64

## New vs used

In [7]:
c = {}
c['new']=0
c['used']=0
new_viewed_by_cond = {}
new_viewed_by_cond['new']=0
new_viewed_by_cond['used']=0
viewd_by_cond = {}
viewd_by_cond['new']=0
viewd_by_cond['used']=0
for x in ds():
    condition_bought = item_data[x['item_bought']]['condition']
    if condition_bought=='not_specified': continue
    conds = [ item_data[i['event_info']]['condition'] for i in x['user_history'] if i['event_type']=='view' ]
    if len(conds)==0: continue
    c[condition_bought]+=1
    new_viewed_by_cond[condition_bought]+=len([None for c in conds if c=='new'])
    viewd_by_cond[condition_bought]+=len(conds)

print(f'Cantidad nuevos: {c["new"]}')

print(f'Cantidad usados: {c["used"]}')
print(f'% usados: {c["used"]/(c["new"]+c["used"])}')
print(f'% vistos nuevos si compro usado: {new_viewed_by_cond["used"]/viewd_by_cond["used"]}')
print(f'% vistos nuevos si compro nuevos: {new_viewed_by_cond["new"]/viewd_by_cond["new"]}')

Cantidad nuevos: 385902
Cantidad usados: 382
% usados: 0.0009889097140963643
% vistos nuevos si compro usado: 0.72053550083672
% vistos nuevos si compro nuevos: 0.9484652925990907


## Check is time sorted

In [8]:
for fn in [TRAIN_DATA, TEST_DATA]:
    for i,x in enumerate(ds()):
        l = [ y['event_timestamp'] for y in x['user_history'] ]
        ls = sorted(l)
        if l!=ls:
            print(i)
            print(l)
            print(ls)
            print(ls==range(len(l)))
            print(l[37])
            print(l[36])
            break